In [1]:
import io
import glob
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFEncryptionError, PDFSyntaxError
from Crypto.Cipher import AES
import os
import sys, getopt
import pandas as pd
import re
from docx import Document
import os

# ConvertMultiple
Files uploaded by the administrator are either PDF or DOC.
All DOC files are converted to DOCX because DOCX is an open standard while DOC is proprietary. docx library was used for that.
The best library to handle pdf files is pdfminer.six, which is a community maintained fork of the original PDFMiner. It is a tool for extracting information from PDF documents focused on getting and analyzing text data.

First step of convertMultiple is to check what files are not converted, files that exist on all_files but not on converted. Secondly the file extension is used to choose on which function the file will go through convertDocx or convertPdf. 
convertPdf may throw some errors in case the files needs a password or it was corrupted, for that we have 2 exceptions

In [1]:
from util import transform_file_to_txt

In [2]:
transform_file_to_txt.convert_doc('../data/teste/pdf/')

In [2]:
transform_file_to_txt.convertmultiple('../data/teste/pdf/', '../data/teste/text/')

In [2]:
os.chdir('/media/mateus/F384-E080/fee/pdf2/')
os.system('lowriter --convert-to docx *.doc')
os.system('lowriter --convert-to docx *.DOC')
os.system('rm *.doc')
os.system('rm *.DOC')

0

In [1]:
#converts pdf, returns its text content as a string
def convertPdf(fname):
    
    pagenums=set()
    output = io.StringIO()
    manager = PDFResourceManager()
    laparams = LAParams()
    password="''"
    
    converter = TextConverter(manager, output, laparams=laparams)
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    
    
    for page in PDFPage.get_pages(infile, pagenums, password = password, check_extractable = False):

        interpreter.process_page(page)
        
    infile.close()
    converter.close()
    text = output.getvalue()
    
    output.close()
    return text 

def convertDocx(fname):
    document = Document(fname)
    for para in document.paragraphs:
        text += str(para.text)+'/n' 
    return text



def convertMultiple(fileDir, txtDir):
    
    all_files = [".".join(f.split(".")[:-1]) for f in os.listdir(fileDir)]
    converted_files = [".".join(f.split(".")[:-1]) for f in os.listdir(txtDir)]
    not_converted_files = [f for f in all_files if f not in converted_files]
    
    files_list = glob.glob(fileDir+"/*")

    
    for file_name in not_converted_files: #iterate through pdfs in pdf directory
        
        full_file_name = [s for s in files_list if file_name in s][0]
        fileExtension = full_file_name.split(".")[-1]
        
        if fileExtension.lower() == "pdf":

            try:
                text = convertPdf(full_file_name)

            except PDFEncryptionError as e :
                command = f"qpdf --password={''} --decrypt {full_file_name} {full_file_name[:-4]}D.pdf;"
                os.system(command)
                text = convertPdf(full_file_name[:-4]+'D.pdf')
            
            except PDFSyntaxError:
                print('PDFSyntaxError')
                next
                
                
        elif fileExtension == "docx":
            
            text = convertDocx(full_file_name)
            
        else:
            print(fileExtension)
            next
            
        textFilename = txtDir+file_name+ ".txt"
        textFile = open(textFilename, "w") 
        
        textFile.write(text)
        textFile.close()


fileDir = "/media/mateus/F384-E080/fee/pdf2/"
txtDir = "/media/mateus/F384-E080/fee/txt2/"
convertMultiple(fileDir, txtDir)

NameError: name 'os' is not defined